In [1]:
ps5_file_name = 'baseline_test_test.csv'

In [2]:
import re
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
 
warnings.filterwarnings("ignore", category=FutureWarning)

TW = 8

M1 = 64
N1 = 64
M2 = 64
N2 = 64

# 阵列分区使用控制信号
seg_arr = False

In [3]:
ps5_sim_results = pd.read_csv(ps5_file_name)

In [4]:
# 参数文件解析
def replace_in_config_file(config_file, key, val):
    with open(config_file, 'r') as f:
        content = f.read()
    
    content = re.sub(r'^{} = (\d+)'.format(key), '{} = {}'.format(key, val), content, flags=re.MULTILINE)
    # print(content)
    
    with open(config_file, 'w') as f:
        f.write(content)
from baseline_int.src.simulator.simulator import Simulator

In [5]:
# 计算baseline能耗
def cal_baseline_energy_util(sweep_data, baseline_e_energy_costs, dram_cost=15.e-3):
    core_energy = (sweep_data['Cycles'] - sweep_data['Memory wait cycles']) * baseline_e_energy_costs.core_dynamic_energy

    breakdown = [core_energy]

    sram_energy = sweep_data['WBUF Read'] * baseline_e_energy_costs.wbuf_read_energy + sweep_data['WBUF Write'] * baseline_e_energy_costs.wbuf_write_energy\
                + sweep_data['IBUF Read'] * baseline_e_energy_costs.ibuf_read_energy + sweep_data['IBUF Write'] * baseline_e_energy_costs.ibuf_write_energy\
                + sweep_data['OBUF Read'] * baseline_e_energy_costs.obuf_read_energy + sweep_data['OBUF Write'] * baseline_e_energy_costs.obuf_write_energy
    breakdown.append(sram_energy)

    dram_energy = (sweep_data['DRAM Read'] + sweep_data['DRAM Write']) * dram_cost
    breakdown.append(dram_energy)

    total_energy = core_energy + sram_energy + dram_energy
    breakdown.append(total_energy)

    return breakdown


def cal_baseline_energy_by_NM(N, M, sweep_data, syn_fn):
    # BitFusion configuration file
    config_file = 'ps5_e_conf_test.ini'  # 定义参数文件
    # replace "a = (\d+)"" in config_file to "a = N"
    replace_in_config_file(config_file=config_file, key='a', val=N)  # 以a输出N，以c输出M
    replace_in_config_file(config_file=config_file, key='c', val=M)
    # N代表PE个数，m代表输入维度？
    batch_size = 100

    # Create simulator object
    verbose = False  # 设置输出信息类型
    baseline_sim = Simulator(config_file, verbose)  # 建立仿真器
    baseline_e_energy_costs = baseline_sim.get_energy_cost(syn_fn)  # 计算能耗

    energy_cost = cal_baseline_energy_util(sweep_data, baseline_e_energy_costs, dram_cost=15.e-3)

    return energy_cost


def cal_baseline_energy(baseline_sim_results, syn_fn):
    energy_table = []  # core_energy, sram_energy, dram_energy, total_energy
    for index, row in baseline_sim_results.iterrows():
        N_energy = row['N']
        M_energy = row['M']
        per_MN_energy_cost = cal_baseline_energy_by_NM(N_energy, M_energy, row, syn_fn)
        energy_table.append(per_MN_energy_cost)
    
    # 转换为 DataFrame
    energy_table = pd.DataFrame(energy_table, columns=['baseline_core_energy', 'baseline_sram_energy', 'baseline_dram_energy', 'baseline_total_energy'])

    # 添加回表格中
    baseline_sim_results = pd.concat([baseline_sim_results, energy_table], axis=1)

    return baseline_sim_results

In [6]:
# 计算PS5能耗
syn_fn = 'systolic_array_synth_ps5.csv'
ps5_sim_results = cal_baseline_energy(ps5_sim_results.reset_index(drop=True), syn_fn)

In [7]:
ps5_sim_results = ps5_sim_results.groupby(['Network'],as_index=False)
ps5_sim_results = ps5_sim_results.agg(np.sum)

In [9]:
ps5_sim_results

,Network,N,M,Max Precision (bits),Min Precision (bits),Layer,Cycles,Memory wait cycles,WBUF Read,WBUF Write,...,in_sparsity_util,out_sparsity_util,best_tiling,TW,Speed,best_order,baseline_core_energy,baseline_sram_energy,baseline_dram_energy,baseline_total_energy
0,CIFAR10SNN,192,192,48,48,conv0/Convolutionconv1/Convolutionconv2/Convol...,133096912,2776,24489034752,70682624,...,0.000000,4.890206,"{'B/b': (1, 100), 'OW/ow': (16, 8), 'OH/oh': (...",48,6,"('OC/oc', 'OH/oh', 'IC/ic', 'B/b', 'OW/ow')('O...",5.644878e+08,2.313991e+10,3.556270e+08,2.406002e+10
1,DVS128Gesture,224,224,56,56,conv0/Convolutionconv1/Convolutionconv2/Convol...,23710144,3816,5276311040,4818944,...,0.000000,5.960816,"{'B/b': (1, 100), 'OW/ow': (16, 8), 'OH/oh': (...",56,7,"('OC/oc', 'OH/oh', 'IC/ic', 'B/b', 'OW/ow')('O...",1.005449e+08,4.985623e+09,7.911484e+07,5.165283e+09
2,ImageNetSNN,512,512,128,128,conv1/Convolutionconv2/Convolutionconv3/Convol...,65323195,9723,13345652736,87883776,...,0.076923,10.852168,"{'B/b': (1, 100), 'OW/ow': (7, 8), 'OH/oh': (7...",128,16,"('IC/ic', 'B/b', 'OC/oc', 'OW/ow', 'OH/oh')('I...",2.770119e+08,1.261043e+10,1.781458e+08,1.306558e+10
